<a href="https://colab.research.google.com/github/KozhevnikovAlexandr/OTALA-/blob/main/training%20neural%20network%20cat-vs-dog%20via%20VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
from google.colab import files
import shutil
import json

from keras import losses
import os
from keras.optimizers import Adam

In [4]:
# Каталог с данными для обучения
train_dir = '/root/train'
# Каталог с данными для проверки
val_dir = '/root/test'
# Каталог с данными для тестирования
test_dir = '/root/val'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 10000
# Количество изображений для проверки
nb_validation_samples = 1000
# Количество изображений для тестирования
nb_test_samples = 1000
#Количество эпох
epochs_count = 5
#Количество нейронов на полносвязном слое
dense_count = 512

In [1]:
#каталоги и файлы для подключения к kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

In [ ]:
#сохранение токена пользователья kaggle
api_token = {"username":"kozhevnikovalexandr","key":"1d2cd622e8b03a7dad939fcc90b811ab"}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#загрузка данных из kaggle
!kaggle competitions download -c dogs-vs-cats

In [ ]:
#распаковка файлов
!mkdir ~/unzipped
!unzip train.zip -d ~/unzipped

In [ ]:
#функция для копирования части изображений в другой каталог
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "cat." + str(i) + ".jpg"), 
                    os.path.join(dest_dir, "cats"))
        shutil.copy2(os.path.join(source_dir, "dog." + str(i) + ".jpg"), 
                   os.path.join(dest_dir, "dogs"))

In [ ]:
#создание каталогов для хранения данных
!mkdir ~/train
!mkdir ~/train/cats
!mkdir ~/train/dogs
!mkdir ~/test
!mkdir ~/test/cats
!mkdir ~/test/dogs
!mkdir ~/val
!mkdir ~/val/cats
!mkdir ~/val/dogs

In [ ]:
#распределение по нужным каталогам
copy_images(1, 10000, '/root/unzipped/train', train_dir)
copy_images(10000, 11000, '/root/unzipped/train', val_dir)
copy_images(11000, 12000, '/root/unzipped/train', test_dir)

In [ ]:
#инициализация готовой части сети
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
#отключение обучения уже обученной части
vgg16_net.trainable = False

In [ ]:
 #создание модели, 
 #добавление полносвязных слоев для классификации по выделенным признакам
  model = Sequential()
  model.add(vgg16_net)
  model.add(Flatten())
  model.add(Dense(dense_count))
  model.add(Activation('relu'))
  model.add(Dropout(0.25))
  model.add(Dense(1))
  model.add(Activation('sigmoid'))

In [ ]:
#компиляция модели
#фунуция ошибок - среднекватратичная 
#оптимизатор Adam
model.compile(loss=losses.mean_squared_error,
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

In [ ]:
#создание генератора, помещаем все пиксели изображения в диапазон от 0 до 255
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
#генератор данных для обучения
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 19998 images belonging to 2 classes.


In [ ]:
#генератор данных для проверки
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.


In [ ]:
#генератор данных для тестиования
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
#обучение модели
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs_count,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
156/156 [==============================] - 2078s 13s/step - loss: 0.2473 - accuracy: 0.5907 - val_loss: 0.1355 - val_accuracy: 0.8188
Epoch 2/10
156/156 [==============================] - 2075s 13s/step - loss: 0.1405 - accuracy: 0.7992 - val_loss: 0.1071 - val_accuracy: 0.8583
Epoch 3/10
156/156 [==============================] - 2074s 13s/step - loss: 0.1155 - accuracy: 0.8385 - val_loss: 0.0965 - val_accuracy: 0.8729
Epoch 4/10
156/156 [==============================] - 2100s 13s/step - loss: 0.1037 - accuracy: 0.8581 - val_loss: 0.0899 - val_accuracy: 0.8833
Epoch 5/10
156/156 [==============================] - 2098s 13s/step - loss: 0.0969 - accuracy: 0.8662 - val_loss: 0.0876 - val_accuracy: 0.8854
Epoch 6/10
156/156 [==============================] - 2093s 13s/step - loss: 0.0907 - accuracy: 0.8798 - val_loss: 0.0845 - val_accuracy: 0.8844
Epoch 7/10
156/156 [==============================] - 2084s 13s/step - loss: 0.0861 - accuracy: 0.8853 - val_loss: 0.0843 - val_ac

In [ ]:
#оценка работы на генераторе для тестиования
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [ ]:
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))